# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Изучим данные, которые находятся в файле `/datasets/data.csv`. Обработаем эти данные и ответим на вопросы, 
- Существует ли зависимость между наличием детей и возвратом кредита в срок?
- Существует ли зависимость между семейным положением и возвратом кредита в срок?
- Существует ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
client_data = pd.read_csv('/datasets/data.csv') 

Посмотрим, насколько большой наш датасет

In [3]:
client_data.shape

(21525, 12)

Видим, что в нашем датасете 21525 строк и 12 столбцов

Посмотрим первые несколько строк

In [4]:
client_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим названия столбцов на адекватность названия для удобной работы в дальнейшем

In [5]:
client_data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Названия оказались весьма красивыми и удобными. Посмотрим теперь общую информацию

In [6]:
client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Получив общую информацию о таблице, стало понятно, что данные в таблице сырые и их надо обработать

### Обработка пропусков

Проверим наш датасет на наличие пустых значений

In [7]:
client_data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения есть в столбцах `days_employed` и `total_income`.

Выберем сначала все строки с пропущенными значениями, затем сгруппируем их по столбцу `income_type`, чтобы понять у кого не указан стаж работы и доход.

In [8]:
nan_values = client_data[client_data['days_employed'].isnull()]
result = nan_values.groupby('income_type').count()
result

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,147,0,147,147,147,147,147,147,147,0,147
компаньон,508,0,508,508,508,508,508,508,508,0,508
пенсионер,413,0,413,413,413,413,413,413,413,0,413
предприниматель,1,0,1,1,1,1,1,1,1,0,1
сотрудник,1105,0,1105,1105,1105,1105,1105,1105,1105,0,1105


Значения отсутствуют у разных категорий граждан, а не только у одной (например, студенты)

Пропущенные значения из двух столбцов связаны между собой. Если у человека не указан стаж, то и доход тоже не указан. Сгруппировав данные о клиентах по типу занятости, заполним пропуски медианным значением для каждой группы

In [9]:
grouped_by_income_type = client_data.groupby(['income_type'])
client_data['total_income'] = grouped_by_income_type.total_income.apply(lambda x: x.fillna(x.median()))

Проверим, все ли пустые значения столбца `total_income` заполнились медианным для каждой группы 

In [10]:
client_data['total_income'].isnull().sum()

0

In [11]:
client_data = client_data.fillna(0) 

Все данные о доходах заполнены, как и требовалось

Посмотрим теперь, нет ли подозртиельных значений в столбце `children`

In [12]:
children = client_data.groupby('children')['children'].count()
children

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Видно, что почему-то у некоторых людей -1 ребенок, а у кого-то сразу 20 детей. Это выглядит крайне подозрительным, 
учитывая, что после 5 детей идёт сразу 20. Здесь явно какая-то ошибка, природа которой не ясна. Удалим строки, где количество детей -1 или 20

In [13]:
client_data = client_data.loc[(client_data['children'] != -1) & (client_data['children'] != 20)] # вместо удаления сделаем просто срез
children = client_data.groupby('children')['children'].count()
children

children
0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Проверим возраст наших потенциальных заёмщиков

In [14]:
age = client_data.groupby('dob_years')['dob_years'].count()

Замечаем, что у целой сотни клиентов в графе возраст почему-то стоит 0 лет. В идеале, можно разбить на кластеры (например, по типу занятости, чтобы точнее заполнить возраст пенсионера, для остальных можно отталкиваться от общего трудового стажа), но в данной задаче, для упрощения, заменим эти нули на средний возраст всех клиентов

In [15]:
mean_age = client_data['dob_years'].mean()
client_data.loc[client_data['dob_years'] == 0, 'dob_years'] = mean_age.astype('int') #делаем срез, указываем столбец, присваиваем средний возраст и приводим к типу int
client_data.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


In [16]:
age = client_data.groupby('dob_years')['dob_years'].count()

Теперь данные в нашем датасете более приятны для анализа. Мы обработали выбросы и некоторые ошибки (как, например, возраст 0 лет). Однако, это еще не все. Трудовой стаж выглядит довольно странно

### Замена типа данных

Скорее всего, трудовой стаж отрицательный, потому что когда его высчитывали и подставляли даты, то перепутали, что от чего отнимать

И дробный он по той причине, что посчитан с точностью до секунд, хоть и выражен в днях.

Поэтому приведем к типу int и домножим на -1

In [17]:
client_data['days_employed'] = client_data['days_employed'].astype('int') *(-1) 
client_data['total_income'] = client_data['total_income'].astype('int')
client_data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,-340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [18]:
age = client_data.loc[client_data['days_employed'] < 0]
print((age['days_employed'] < -300000).value_counts())

True    3431
Name: days_employed, dtype: int64


Видим, что для 3431 записи стаж отрицательный, более того, менее трёхсот тысяч. Здесь точно не просто ошибка в знаке. Возможно, это стаж в часах. Тогда поделим эти результаты на 24, чтобы привести к дням и домножим на -1

In [19]:
HOURS_IN_DAY = 24
client_data.loc[(client_data['days_employed'] < -300000), 'days_employed'] /= HOURS_IN_DAY * (-1)
client_data['days_employed'] = client_data['days_employed'].astype('int')

client_data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Данные успешно преобразованы, теперь стаж не вызывает таких больших вопросов, как ранее. Также, ежемесячный доход преобразован к целому типу, потому что при таких больших значениях дробной частью можно пренебречь

### Обработка дубликатов

Для начала приведём все к нижнему регистру. Воспользуемся методом `str.lower()`, который относится к `Series`, а значит, придется каждый столбец, где содержится строковый тип данных приводить к нижнему регистру отдельно

In [20]:
client_data['education'] = client_data['education'].str.lower() 
client_data['family_status'] = client_data['family_status'].str.lower()
client_data['gender'] = client_data['gender'].str.lower()
client_data['income_type'] = client_data['income_type'].str.lower()
client_data['purpose'] = client_data['purpose'].str.lower()

Теперь посмотрим, имеются ли дубликаты

In [21]:
client_data.duplicated().sum()

71

Видим, что в нашей таблице имеется 71 дубликат. Скорее всего, дубликаты появились из-за того, что данные о заёмщиках вносили в разном регистре. Избавимся от дубликатов методом `drop_duplicates`

In [22]:
client_data = client_data.drop_duplicates()

Наши данные стали еще более чистыми и еще более пригодными для обработки. Теперь все в одном регистре и без дубликатов

### Лемматизация

Напишем функцию, которая будет лемматизировать причины для получения кредита

In [23]:
m = Mystem()
def get_lemmatized_purpose(data):
    '''Функция принимает на вход список причин взятия кредита, каждую причину лемматизирует
    и добавляет в список lemmatized_purpose. Возвращается список лемматизированных причин взятия кредита'''
    
    lemmatized_purpose = []
    for line in data:
        lemmatized_purpose.append(m.lemmatize(line))
    return lemmatized_purpose

In [24]:
client_data['purpose'] = get_lemmatized_purpose(client_data['purpose'])

Получив список лемматизированных причин, посмотрим все значения и их количество

In [25]:
client_data['purpose'].value_counts()

[автомобиль, \n]                                          967
[свадьба, \n]                                             790
[на,  , проведение,  , свадьба, \n]                       763
[сыграть,  , свадьба, \n]                                 760
[операция,  , с,  , недвижимость, \n]                     672
[покупка,  , коммерческий,  , недвижимость, \n]           658
[покупка,  , жилье,  , для,  , сдача, \n]                 649
[операция,  , с,  , жилье, \n]                            647
[операция,  , с,  , коммерческий,  , недвижимость, \n]    645
[жилье, \n]                                               641
[покупка,  , жилье, \n]                                   640
[покупка,  , жилье,  , для,  , семья, \n]                 637
[недвижимость, \n]                                        631
[строительство,  , собственный,  , недвижимость, \n]      628
[операция,  , со,  , свой,  , недвижимость, \n]           623
[строительство,  , жилой,  , недвижимость, \n]            620
[строите

Для удобства работы в дальнейшем сделаем словарь, где ключом будет цель кредита, а значениями - всевозможные интерпретации одной и той же цели

In [26]:
dict_purpose = {'ремонт': 'ремонт', 'автомобиль': ('автомобиль', 'машина'), 'свадьба': 'свадьба', 
               'недвижимость': ('недвижимость', 'жилье', 'квартира'), 'образование': 'образование'}
dict_purpose

{'ремонт': 'ремонт',
 'автомобиль': ('автомобиль', 'машина'),
 'свадьба': 'свадьба',
 'недвижимость': ('недвижимость', 'жилье', 'квартира'),
 'образование': 'образование'}

Основные причины, чтобы взять кредит - это покупка недвижимости, покупка автомобиля, получение образования, свадьба, ремонт

### Категоризация данных

Разобьем наши данные на категории по целям кредита

In [27]:
dict_purpose = {'ремонт': ['ремонт'], 'автомобиль': ['автомобиль', 'машина'], 'свадьба': ['свадьба'], 
               'недвижимость': ['жилье', 'недвижимость', 'квартира'], 'образование': ['образование']}


def get_categorized_purpose(lemmatized_purpose):
    '''Функция принимает лемматизизрованную причину займа и возвращает категоризированную причину, используя словарь. 
    Если причина не найдена в словаре, функция возвращает значение иная причиная'''
    
    lemmatized_purpose = lemmatized_purpose['purpose']
    
    for lemma in lemmatized_purpose:
        for key, list_values in dict_purpose.items():
            if lemma in list_values:
                return key
    return 'иная причина'

Применим функцию для категоризации данных и запишем в отдельный столбец `categorized purpose`

In [28]:
client_data['categorized_purpose'] = client_data.apply(get_categorized_purpose, axis=1)

Проверим, все ли строки получили определенную категорию. Для этого посмотрим, сколько значений `иная причина` получилось

In [29]:
check = client_data.loc[client_data['categorized_purpose'] == 'иная причина']
check.size

0

In [30]:
client_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,categorized_purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,"[дополнительный, , образование, \n]",образование
4,0,14177,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,"[сыграть, , свадьба, \n]",свадьба
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,"[покупка, , жилье, \n]",недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,"[операция, , с, , жилье, \n]",недвижимость
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,"[образование, \n]",образование
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,"[на, , проведение, , свадьба, \n]",свадьба
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,"[покупка, , жилье, , для, , семья, \n]",недвижимость


Все причины взятия кредита разделены на 5 конкретных категорий (недвижимость, ремонт, свадьба, автомобиль, образование). Ни одна причина не отнесена к разделу *иная причина*

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Составим сводную таблицу с группировкой по количеству детей и отобразим в ней количестов должников, общее количество людей и долю должников от общего числа людей, принадлежащих одной категории

In [31]:
data_pivot_children = client_data.pivot_table(index='children', values='debt', aggfunc = ['sum', 'count', 'mean'])
data_pivot_children

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,0.075438
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


### Вывод

Видим, что всего было 9 заёмщиков, кто имеет 5 детей и все они вернули кредит вовремя. Среди остальных же, больше всего вероятность своевременного возврата кредита у тех, у кого детей нет. Далее следует группа людей, у которых 3 ребёнка, что довольно интересно. Согласно получившимся данным, те, у кого троей детей с большей вероятностью вернут кредит в срок, чем те, у кого детей 1, 2 или 4

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Составим сводную таблицу с группировкой по семейному положению и отобразим в ней количестов должников, общее количество людей и долю должников от общего числа людей, принадлежащих одной категории

In [32]:
data_pivot_family_status = client_data.pivot_table(index='family_status', values='debt', aggfunc = ['sum', 'count', 'mean'])
data_pivot_family_status

,sum,count,mean
,debt,debt,debt
family_status,,,
в разводе,84,1189,0.070648
вдовец / вдова,63,951,0.066246
гражданский брак,385,4134,0.093130
женат / замужем,927,12261,0.075606
не женат / не замужем,273,2796,0.097639


Как видим, те, кто не женат / не замужем и те, кто в гражданском браке, с большей вероятностью будут иметь задолженности по кредиту. Меньше всего должников среди овдовевших

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Воспользуемся методом `qcut` и разобьем наши данные на 5 квантилей

In [33]:
quantiles_5 = pd.qcut(x=client_data['total_income'], q=5)
quantiles_5[0:1]

0    (214604.0, 2265604.0]
Name: total_income, dtype: category
Categories (5, interval[float64]): [(20666.999, 98514.0] < (98514.0, 132113.0] < (132113.0, 161380.0] < (161380.0, 214604.0] < (214604.0, 2265604.0]]

Получили 5 промежутков. Добавим их в нашу таблицу как отдельный столбец, чтобы потом вывести сводную таблицу

In [34]:
client_data['total_income_by_5_groups'] = quantiles_5

In [35]:
data_pivot_income = client_data.pivot_table(index='total_income_by_5_groups', values='debt', aggfunc = ['sum', 'count', 'mean'])
data_pivot_income

,sum,count,mean
,debt,debt,debt
total_income_by_5_groups,,,
"(20666.999, 98514.0]",344,4267,0.080619
"(98514.0, 132113.0]",358,4266,0.083919
"(132113.0, 161380.0]",373,4266,0.087436
"(161380.0, 214604.0]",358,4266,0.083919
"(214604.0, 2265604.0]",299,4266,0.070089


Как видим, самая низкая доля должников оказаалась у людей с доходом >214604, то есть с самым большим доходом. В остальных группах наблюдается примерно одинаковая вероятность задолженности по кредиту. 

- Как разные цели кредита влияют на его возврат в срок?

Посмотрим, сколько заемщиков в каждой категории по целям кредита и сколько из них должников.

In [39]:
data_pivot_purpose = client_data.pivot_table(index='categorized_purpose', values='debt', aggfunc = ['sum', 'count', 'mean'])
data_pivot_purpose

,sum,count,mean
,debt,debt,debt
categorized_purpose,,,
автомобиль,400,4279,0.093480
недвижимость,745,10147,0.073421
образование,369,3988,0.092528
ремонт,35,604,0.057947
свадьба,183,2313,0.079118


Люди, которые взяли кредит с целью сделать ремонт, с большей вероятностью выплатят кредит в срок. Самая высокая доля должников оказалась в категориях с целью получения образования и покупки автомобиля. 

### Вывод

В ходе анализа информации о заемщиках, эту информацию пришлось сначала обработать. Заполнить пропущенные значения о доходе средним для каждой группы, обработать аномальные данные в столбце `количество детей`, где у кого-то был -1 ребенок, а у кого-то 20. Обработаны данные заемщиков, у которых в столбце `возраст` было значение 0 лет. И дополнительно был поправлен стаж, т.к. у многих он был отрицательным, а у кого-то сотни лет. 

После обработки пропусков и аномальных значений, были преобразованы типы данных. Дробные числа были заменены на целые в столбцах `доход` и `стаж`. После этого нужно было обработать повторяющиеся значения - дубликаты. Для этого я привёл всю текстовую информацию к нижнему регистру, посмотрел, имеются ли дубликаты (а они имелись) и удалил их.

Далее я выявил леммы в столбце `цель кредита` для того, чтобы определить всевозможные цели кредита по категориям. Таких категорий получилось 5 и удобно было их обработать с помощью словаря. Интересно, что все данные из таблицы можно разделить на эти 5 конкретных категорий и не оказалось ни одной цели кредита, которая была бы отнесена к *иным причинам*

И вот, когда данные были обработаны, я начал искать, будут ли влиять определенные факторы на возврат кредита в срок. Например, семейное положение, количество детей, уровень дохода, цель кредита. Использовав сводные таблицы, всё сразу стало наглядно и понятно. Применив методы `sum()`, `count()`, `mean()` я увидел количество должников, в зависимости от группировки по нужному фактору, количество заёмщиков всего и долю тех, кто не возвращает кредит в срок. Также для определения зависимости от уровня дохода я использовал метод `qcut()` и разбил всех заёмщиков на 5 равных квантилей.

Что же получилось в итоге? А по итогу оказалось, что зависимость есть. Больше всего вероятность своевременного возврата кредита у тех, у кого детей нет; те, кто не женат / не замужем и те, кто в гражданском браке, с большей вероятностью будут иметь задолженности по кредиту, а меньше всего должников среди овдовевших; самая низкая доля должников оказаалась у людей с самым большим доходом; самая высокая доля должников оказалась в категориях с целью получения образования и покупки автомобиля, а люди, взявшие кредит н ремонт, с большей вероятностью выплатят его в срок.